# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [42]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import torch.nn.functional as torch_func

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [43]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [44]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,10,5)
        self.conv2=nn.Conv2d(10,20,3)
        self.func1 = nn.Linear(20*10*10,500)
        self.func2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0)
        out = torch_func.relu(self.conv1(x))
        out = torch_func.max_pool2d(out, 2, 2)
        out = torch_func.relu(self.conv2(out))
        out = out.view(in_size,-1)
        out = torch_func.relu(self.func1(out))
        out = torch_func.log_softmax(self.func2(out),dim=1)
        return out


import torch.optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 让torch判断是否使用GPU
model = SimpleNet().to(DEVICE)
#criterion = LossCriterion()
optimizer = torch.optim.Adam(model.parameters())

Next, we can start to train and evaluate!

In [52]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = torch_func.nll_loss(output, labels)
        loss.backward()
        optimizer.step()
    model.eval()
    test_loss = 0
    correct_num = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            test_loss += torch_func.nll_loss(output, labels, reduction='sum').item() # 损失求和
            pred = output.max(1, keepdim=True)[1]                                    # 找到概率最大的类别
            correct_num += pred.eq(labels.view_as(pred)).sum().item()

    test_loss =test_loss/len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct_num, len(test_loader.dataset),
        100. * correct_num / len(test_loader.dataset)))
    



Test set: Average loss: 0.0271, Accuracy: 9903/10000 (99%)


Test set: Average loss: 0.0319, Accuracy: 9897/10000 (99%)


Test set: Average loss: 0.0323, Accuracy: 9895/10000 (99%)


Test set: Average loss: 0.0411, Accuracy: 9876/10000 (99%)


Test set: Average loss: 0.0360, Accuracy: 9888/10000 (99%)


Test set: Average loss: 0.0357, Accuracy: 9893/10000 (99%)


Test set: Average loss: 0.0521, Accuracy: 9861/10000 (99%)


Test set: Average loss: 0.0366, Accuracy: 9890/10000 (99%)


Test set: Average loss: 0.0386, Accuracy: 9892/10000 (99%)


Test set: Average loss: 0.0381, Accuracy: 9900/10000 (99%)



#### Q5:
Please print the training and testing accuracy.